In [ ]:
import numpy as np
import pandas as pd

In [ ]:
BLOCKS_POPULATION_FILE = './data/blocks_population.txt'
CONFIG = './data/problem_config.txt'

In [ ]:
population = pd.read_csv(BLOCKS_POPULATION_FILE, header=None)
population

In [ ]:
config = pd.read_json(CONFIG, typ='series')
config

In [ ]:
population.rename(index=lambda x: x+0.5, columns=lambda x: x+0.5, inplace=True)
population

In [ ]:
GRIDS = population.shape[0]
GRIDS

In [ ]:
class Neighbor:
    def __init__(self, x, y, population):
        self.x = x
        self.y = y
        self.population = population
        self.bts = None

    def __repr__(self):
        return f'({self.x}, {self.y}, {self.population})'

    def set_bts(self, bts):
        self.bts = bts

    def calculate_bandwidth_per_user(self):
        return self.bts.calculate_bandwidth(self) / self.population

    def calculate_satisfaction(self):
        bpu = self.calculate_bandwidth_per_user()
        satisfaction = 0 if bpu < config['user_satisfaction_levels'][0] else config['user_satisfaction_levels'][-1]

        for i in range(1, len(config['user_satisfaction_levels']) - 1):
            if config['user_satisfaction_levels'][i - 1] < bpu < config['user_satisfaction_levels'][i]:
                satisfaction = config['user_satisfaction_levels'][i - 1]
                break

        return satisfaction * self.population

In [ ]:
list_of_neighbors = []
for i in population.index:
    for j in population.columns:
        list_of_neighbors.append(Neighbor(i, j, population.loc[i, j]))

list_of_neighbors

In [ ]:
class BTS:
    def __init__(self, x, y, bandwidth):
        self.x = x
        self.y = y
        self.bandwidth = bandwidth
        self.cost = config['tower_construction_cost'] + \
            config['tower_maintenance_cost'] * bandwidth
        self.neighbors = set()

    def __repr__(self):
        return f'BTS({self.x}, {self.y}, {self.bandwidth})'

    def add_neighbor(self, neighbor):
        self.neighbors.add(neighbor)

    def calculate_bandwidth_prime(self, neighbor):
        return (neighbor.population * self.bandwidth) / sum(n.population for n in self.neighbors)

    def calculate_bandwidth(self, neighbor):
        diff = np.array((neighbor.x, neighbor.y)) - np.array((self.x, self.y))
        cov_ty_bx = np.exp(-0.5 * diff *
                           np.linalg.inv(np.array([[8, 0], [0, 8]])) * diff.T)
        return cov_ty_bx * self.calculate_bandwidth_prime(neighbor)

    def remove_neighbor(self, neighbor):
        self.neighbors.remove(neighbor)

In [ ]:
class Chromosome:
    def __init__(self, number_of_bts, neighbors):
        self.number_of_bts = number_of_bts
        self.neighbors = neighbors
        self.bts = []

    def initialize(self, number_of_bts):
        for __ in range(number_of_bts):
            self.bts.append(BTS(np.random.uniform(0, GRIDS), np.random.uniform(0, GRIDS), np.random.uniform(
                config['user_satisfaction_levels'][0], config['user_satisfaction_levels'][-1])))

        for neighbor in self.neighbors:
            bts = np.random.choice(self.bts)
            bts.add_neighbor(neighbor)
            neighbor.set_bts(bts)

    def mutate(self):
        # number of bts
        self.number_of_bts = np.random.randint(1, len(self.neighbors)+1)

        # bts
        if len(self.bts) < self.number_of_bts:
            self.initalize(self.number_of_bts - len(self.bts))
        bts = np.random.choice(self.bts, self.number_of_bts, replace=False)

        u = np.random.normal(0, 1)

        std_dev_x_bts = np.std([b.x for b in bts]) * \
            np.exp(u/np.sqrt(len(bts)))
        std_dev_y_bts = np.std([b.y for b in bts]) * \
            np.exp(u/np.sqrt(len(bts)))
        std_dev_bandwidth_bts = np.std(
            [b.bandwidth for b in bts]) * np.exp(u/np.sqrt(len(bts)))

        for b in bts:
            # location and bandwidth
            b.x += np.random.normal(b.x, std_dev_x_bts)
            b.x = max(b.x, 0)
            b.x = min(b.x, GRIDS)

            b.y += np.random.normal(b.y, std_dev_y_bts)
            b.y = max(b.y, 0)
            b.y = min(b.y, GRIDS)

            b.bandwidth += np.random.normal(b.bandwidth, std_dev_bandwidth_bts)
            b.bandwidth = max(
                b.bandwidth, config['user_satisfaction_levels'][0])
            b.bandwidth = min(
                b.bandwidth, config['user_satisfaction_levels'][-1])

        # neighbors
        for neighbor in self.neighbors:
            p_m = np.random.uniform(0, 1)
            if neighbor.bts not in bts:
                random_bts = np.random.choice(bts)
                neighbor.set_bts(random_bts)
                random_bts.add_neighbor(neighbor)

            elif p_m <= 0.5:
                neighbor.bts.remove_neighbor(neighbor)
                random_bts = np.random.choice(bts)
                neighbor.set_bts(random_bts)
                random_bts.add_neighbor(neighbor)

        self.bts = bts

    def recombinate(self, other):
        def get_indices(list, element): return [
            ind for ind, value in enumerate(list) if element in value.neighbors]

        cross_over_point_self = np.random.randint(1, self.number_of_bts-1)
        cross_over_point_other = np.random.randint(1, other.number_of_bts-1)

        bts_offspring1 = self.bts[0:cross_over_point_self] + \
            other.bts[cross_over_point_other:]
        bts_offspring2 = other.bts[0:cross_over_point_other] + \
            self.bts[cross_over_point_self:]

        for neighbor in self.neighbors:
            list_of_index = bts_offspring1.get_indices(
                bts_offspring1, neighbor.bts)
            if len(list_of_index) == 0:
                neighbor.set_bts(np.random.choice(bts_offspring1))
                neighbor.bts.add_neighbor(neighbor)

            elif len(list_of_index) > 1:
                for ind in range(1, list_of_index):
                    bts_offspring1[ind].remove_neighbor(neighbor)

        for neighbor in other.neighbors:
            list_of_index = bts_offspring2.get_indices(
                bts_offspring2, neighbor.bts)
            if len(list_of_index) == 0:
                neighbor.set_bts(np.random.choice(bts_offspring2))
                neighbor.bts.add_neighbor(neighbor)

            elif len(list_of_index) > 1:
                for ind in range(1, list_of_index):
                    bts_offspring2[ind].remove_neighbor(neighbor)

        offspring1, offspring2 = Chromosome(len(bts_offspring1), self.neighbors), Chromosome(
            len(bts_offspring2), other.neighbors)
        offspring1.bts = bts_offspring1
        offspring2.bts = bts_offspring2

        return offspring1, offspring2

    def calcutaion_fitness(self):
        X, Y = 1.5, 1
        satisfaction = sum(neighbor.calculate_satisfaction()
                           for neighbor in self.neighbors)
        cost = sum(b.cost for b in self.bts)
        return X*(satisfaction) + Y*(cost)

In [ ]:
class Evolutionary_Algorithm:
    def __init__(self, neighbors, population_size=50, max_generation=200, p_mutation=0.1, p_crossover=0.1):
        self.population_size = population_size
        self.max_generation = max_generation
        self.neighbors = neighbors
        self.p_mutation = p_mutation
        self.p_crossover = p_crossover
        self.population = []
        self.parent_pool = []
        self.offsprings = []
        self.best = None

    def initialize(self):
        self.population.clear()

        for __ in range(self.population_size):
            number_of_bts = np.random.randint(1, len(self.neighbors)+1)
            chromosome = Chromosome(number_of_bts, self.neighbors)
            chromosome.initialize()
            self.population.append(chromosome)

    def mutation(self):
        for chromosome in self.offsprings:
            if np.random.uniform(0, 1) <= self.p_mutation:
                chromosome.mutate()

    def recombination(self, parent_pair):
        for ch_1, ch_2 in parent_pair:
            if np.random.uniform(0, 1) <= self.p_crossover:
                offspring_1, offspring_2 = ch_1.recombinate(ch_2)
                self.offsprings.append(offspring_1)
                self.offsprings.append(offspring_2)

    def tournament_selection(self):
        for __ in range(self.population_size):
            tournament = np.random.choice(self.population, 5, replace=True)
            self.parent_pool.append(
                max(tournament, key=lambda x: x.calcutaion_fitness()))

    def generate_next_generation(self):
        self.parent_pool.clear()
        self.offsprings.clear()
        self.tournament_selection()

        parent_pair = []
        np.random.shuffle(self.parent_pool)
        for i in range(0, len(self.parent_pool), 2):
            parent_pair.append((self.parent_pool[i], self.parent_pool[i+1]))

        self.recombination(parent_pair)
        self.mutation()

        self.population += self.offsprings
        self.population = sorted(self.population, key=lambda x: x.calcutaion_fitness(
        ), reverse=True)[:self.population_size]